Atalov S.

Fundamentals of Machine Learning and Artificial Intelligence

# Gradient Boosting Classifier Implementation
---

In [ ]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Load and prepare the data
titanic_train = 'https://raw.githubusercontent.com/lobachevksy/teaching/main/titanic/train.csv'
data = pd.read_csv(titanic_train)

data.drop(columns=['PassengerId', 'Cabin', 'Ticket', 'Name'], inplace=True)

data['Age'].fillna(data['Age'].median(), inplace=True)

data['Sex'] = data['Sex'].map({'male': 1, 'female': 0}).astype(int)

data = pd.get_dummies(data, columns=['Embarked'])

In [ ]:
data

In [ ]:
# Select features and target
X = data.copy()
y = X.pop('Survived')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Implementation

$L(y,\hat{y})=−ylog(\hat{y})−(1−y)log(1−\hat{y})$

In [ ]:
def entropy_loss_gradient(y, p):
    """ Entropy loss """
    p = np.clip(p, 1e-15, 1 - 1e-15)
    return -(y / p) + (1 - y) / (1 - p)

In [ ]:
def sigmoid(z):
    """ Sigmoid function. """
    # TODO
    pass

In [ ]:
n_estimators = 100
learning_rate = 0.5
min_samples_split = 2
max_depth = 2

In [ ]:
# Initialize trees
trees = []
for _ in range(n_estimators):
    trees.append(
        DecisionTreeRegressor(
            max_depth=max_depth, min_samples_split=min_samples_split
        )
    )

# Initial predictions as average
y_hat = np.full_like(y_train, fill_value=np.mean(y_train, axis=0))

In [ ]:
for tree in trees:
    grad = -entropy_loss_gradient(y_train, y_hat)
    
    tree.fit(X_train, grad)
    
    h = tree.predict(X_train)
    
    y_hat = y_hat + learning_rate * h

In [ ]:
def predict(X, trees, learning_rate):
    y_hat = np.array([])
    for tree in trees:
        h = tree.predict(X)
        h = learning_rate * h
        y_hat = h if not y_hat.any() else y_hat + h
        
    y_hat = np.clip(y_hat, 1e-30, 1 - 1e-30)
    
    probabilities = sigmoid(y_hat)
    y_hat = (probabilities > 0.5).astype("int")
    return y_hat

In [ ]:
predict(X_test, trees, learning_rate)

In [ ]:
accuracy_score(y_test, y_pred)